In [110]:
from ipywidgets import Layout, Button, VBox, HBox, Label, Checkbox, interactive
from IPython.display import display
import sys
sys.path.append(r'c:\users\thech\anaconda3\lib\site-packages')
from pyvis.network import Network
import networkx as nx

def make_Networkx_graph(nodes_file, edges_file, group_colors, files_have_header=True):
    net = nx.Graph()
    
    # Add nodes
    with open(nodes_file, 'r') as f:
        if files_have_header:
            next(f)
        
        for line in f:
            line = line.strip()
            # Skip blank or comment lines
            if not line or line.startswith('#'):
                continue 
            
            line_split = line.split('\t')
            ID = line_split[0]
            label = line_split[0]
            title = line_split[2]
            color = group_colors[line_split[1]]
            
            net.add_node(ID, label=label, title=title, color=color)
    
    # Add edges
    with open(edges_file, 'r') as f:
        if files_have_header:
            next(f)
        
        for line in f:
            line = line.strip()
            # Skip blank or comment lines
            if not line or line.startswith('#'):
                continue 
            
            line_split = line.split('\t')
            source = line_split[0]
            to = line_split[1]
            value = line_split[2]
            title = line_split[2]
            hidden = False
            physics = True
            net.add_edge(source, to, value=value, title=title, hidden=hidden, physics=physics)
    
    #print(list(net.edges))
    return net

def updatePlot(**edges_2_remove):
    #print(edges_2_remove)
    for edge in list(nx_graph.edges()):
        e = nx_graph.edges[edge]
        if edges_2_remove[checkbox_values[e["value"]]]:
            nx_graph.edges()[(edge[0], edge[1])]['hidden'] = True
            nx_graph.edges()[(edge[0], edge[1])]['physics'] = False
        else:
            nx_graph.edges()[(edge[0], edge[1])]['hidden'] = False
            nx_graph.edges()[(edge[0], edge[1])]['physics'] = True
    net = Network("700px", "1400px", notebook=True, heading="")
    net.from_nx(nx_graph)
    return net.show("Network.html")

# Returns "value":"(count) value"
def extract_edge_values(nx_graph):
    edge_values = [nx_graph.edges[edge]["value"] for edge in list(nx_graph.edges())] # Get values from edges 
    edge_values_counts = [[v, edge_values.count(v)] for v in set(edge_values)] # Count number of times each value occurs
    return {x:"({1}) {0}".format(x, y) for x, y in sorted(edge_values_counts, key=lambda x: x[1], reverse=True)} # Sort values by count and return value+count string


nx_graph = make_Networkx_graph(nodes_file = "combined_HostSymbiont.magi_gene_results.filtered.RHEA_Annots.txt", 
             edges_file = "combined_HostSymbiont.magi_gene_results.filtered.RHEA_2_RHEA_network_edges.txt", 
             files_have_header=True, group_colors={"Symbiont":"#00ff1e", "Host":"#dd4b39", "Both":"#162347"})

checkbox_values = extract_edge_values(nx_graph)

box_layout = Layout(overflow_y='auto',
                    border='3px solid black',
                    width='',
                    height='600px',
                    flex_flow='column',
                    display='block')

checkboxes = [Checkbox(value=False, description=label, continuous_update=True) for label in list(checkbox_values.values())]
carousel = VBox(children=checkboxes, layout=box_layout)
updatePlot(**{c:False for c in list(checkbox_values.values())}) # Force first update
widget=interactive(updatePlot, **{c.description: c.value for c in checkboxes})
controls = HBox(widget.children[:-1], layout = box_layout)
output = widget.children[-1]
display(VBox([controls, output], indent=False))